![Work Work](../media/image/work_work.gif)



[**WORK WORK  . . .**](https://www.youtube.com/watch?v=eO6OaVnbqaY)

<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Inicialização de Bibliotecas, Pastas e Foco da Camera</p>

In [3]:
import cv2
import numpy as np
import os
import time
import data_processing as dp
from collections import deque

os.makedirs("../data", exist_ok=True)
os.makedirs("../data/raw", exist_ok=True)

for i in range(8):
    os.makedirs(f"../data/raw/{i}", exist_ok=True)

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.25)
cap.set(cv2.CAP_PROP_EXPOSURE, -1)

cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)
cap.set(cv2.CAP_PROP_FOCUS, 10)

print("Foco Ajustado")

Foco Ajustado


<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Bloco Principal</p>

In [2]:
cap = cv2.VideoCapture(r'../media/.bin/sample.mp4')
division = 16 # --------> Division

while True:

    ret,frame_original = cap.read()

    if not ret:
        break

    frame_original = dp.mirroring(frame_original)
    frame = frame_original.copy()
    frame = dp.gray_scale(frame)
    frame = dp.background_subtraction(cam_frame=frame,start_time=0.1,limiar=15)
    sample,frame = dp.down_sampling(frame,division=division)
    frame = dp.binarization(frame,limiar=5)
    sample = dp.binarization(sample,limiar=5)
    bbox_points,bb_frame = dp.bounding_box(frame,frame_vizualizer=frame_original,division=division)
    sample = dp.centralize(sample, bbox_points=bbox_points)
    sample = dp.dilate(sample,kernel=np.ones(shape=(3,3)))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    elif cv2.waitKey(1) & 0xFF == ord('1'):
        dp.background_subtraction.background = dp.gray_scale(frame_original.copy())
        frame = np.full_like(frame_original,255)
        print('Background reset')

    cv2.imshow('frame',frame)
    cv2.imshow('Bounding Box Frame',bb_frame)
    cv2.imshow('sample',sample)

cap.release()
cv2.destroyAllWindows()

<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Bloco Video REC
</p>

In [5]:
gravador = cv2.VideoWriter('../media/.bin/asd.avi',fourcc=cv2.VideoWriter_fourcc(*'XVID'),fps=30,frameSize=(640,480))
cam = cv2.VideoCapture(0)
rec = False
while True:
    ret,original_frame = cam.read()
    original_frame = dp.mirroring(original_frame)

    if not ret:
        break
    
    frame = original_frame.copy()
    frame = dp.gray_scale(frame)
    frame = dp.background_subtraction(cam_frame=frame,start_time=3,limiar=15)
    vector,frame = dp.down_sampling(frame,division=16)
    frame = dp.binarization(frame,limiar=5)
    vector = dp.binarization(vector,limiar=5)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    elif cv2.waitKey(1) & 0xFF == ord('1'):
        dp.background_subtraction.background = dp.gray_scale(original_frame.copy())
        frame_original = np.full_like(original_frame,255)
    elif cv2.waitKey(1) & 0xFF == ord('2'):
        rec = True
        print('gravandooo...')

    if rec:
        frame = np.stack([frame]*3, axis=-1)
        gravador.write(frame)


    cv2.imshow('Original',original_frame)
    cv2.imshow('Visualizer',frame)

cam.release()
gravador.release()
cv2.destroyAllWindows()

gravandooo...
